In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchsummary

import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
batch_size = 32
learning_rate = 0.001
notebookName = "DL_Base"
nepochs = 100
max_patience_count = 100

In [4]:

PATH = Path(f"./models/{notebookName}")
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            print(num_files)
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0
num_files

0

In [5]:
path = "E:\datasets\[programmers]\challenges_849"

# Read Datasets
## Read train

In [6]:
df_train = pd.read_csv(os.path.join(path, "train.csv"))
print(df_train.shape)
df_train.head(5)

(6000, 3)


,userID,jobID,applied
0,fe292163d06253b716e9a0099b42031d,15de21c670ae7c3f6f3f1f37029303c9,0
1,6377fa90618fae77571e8dc90d98d409,55b37c5c270e5d84c793e486d798c01d,0
2,8ec0888a5b04139be0dfe942c7eb4199,0fcbc61acd0479dc77e3cccc0f5ffca7,1
3,f862b39f767d3a1991bdeb2ea1401c9c,3b5dca501ee1e6d8cd7b905f4e1bf723,0
4,cac14930c65d72c16efac2c51a6b7f71,287e03db1d99e0ec2edb90d079e142f3,0


## Read test

In [7]:
df_test = pd.read_csv(os.path.join(path, "test_job.csv"))
print(df_test.shape)
df_test.head(5)

(2435, 2)


,userID,jobID
0,ebaee1af0c501f22ddfe242fc16dae53,352407221afb776e3143e8a1a0577885
1,9ab05403ac7808cbfba3da26665f7a9c,96b9bff013acedfb1d140579e2fbeb63
2,33349e909eba71677299d2fc97e158b7,58d4d1e7b1e97b258c9ed0b37e02d087
3,ac985a9db5faeb44c94a334430ccc241,ccb0989662211f61edae2e26d58ea92f
4,d41e0e6f6f1e29098d9d152511503ab2,4a213d37242bdcad8e7300e202e7caa4


## company

In [8]:
df_company = pd.read_csv(os.path.join(path, "job_companies.csv"))
print(df_company.shape)
df_company.head(5)

(733, 3)


,companyID,jobID,companySize
0,00411460f7c92d2124a67ea0f4cb5f85,e5f6ad6ce374177eef023bf5d0c018b6,NaN
1,1905aedab9bf2477edc068a355bba31a,185e65bc40581880c4f2c82958de8cfe,11-50
2,89fcd07f20b6785b92134bd6c1d0fa42,0537fb40a68c18da59a35c2bfe1ca554,101-200
3,acf4b89d3d503d8252c9c4ba75ddbf6d,b7ee6f5f9aa5cd17ca1aea43ce848496,1-10
4,fb7b9ffa5462084c5f4e7e85a093e6d7,efe937780e95574250dabe07151bdc23,NaN


## jobtag

In [9]:
df_jobtag = pd.read_csv(os.path.join(path, "job_tags.csv"))
print(df_jobtag.shape)
df_jobtag.head(5)

(3477, 2)


,jobID,tagID
0,320722549d1751cf3f247855f937b982,d38901788c533e8286cb6400b40b386d
1,e744f91c29ec99f0e662c9177946c627,3948ead63a9f2944218de038d8934305
2,e744f91c29ec99f0e662c9177946c627,0e095e054ee94774d6a496099eb1cf6a
3,e820a45f1dfc7b95282d10b6087e11c0,7d771e0e8f3633ab54856925ecdefc5d
4,e820a45f1dfc7b95282d10b6087e11c0,6c8dba7d0df1c4a79dd07646be9a26c8


## tag

In [10]:
df_tag = pd.read_csv(os.path.join(path, "tags.csv"))
print(df_tag.shape)
df_tag.head(5)

(887, 2)


,tagID,keyword
0,602d1305678a8d5fdb372271e980da6a,Amazon Web Services(AWS)
1,e3251075554389fe91d17a794861d47b,Tensorflow
2,a1d50185e7426cbb0acad1e6ca74b9aa,Docker
3,884d79963bd8bc0ae9b13a1aa71add73,Git
4,4122cb13c7a474c1976c9706ae36521d,Python


## usertag

In [11]:
df_usertag = pd.read_csv(os.path.join(path, "user_tags.csv"))
print(df_usertag.shape)
df_usertag.head(5)

(17194, 2)


,userID,tagID
0,e576423831e043f7928d9ac113abbe6f,82c2559140b95ccda9c6ca4a8b981f1e
1,e576423831e043f7928d9ac113abbe6f,2ba8698b79439589fdd2b0f7218d8b07
2,e576423831e043f7928d9ac113abbe6f,351b33587c5fdd93bd42ef7ac9995a28
3,e576423831e043f7928d9ac113abbe6f,81e5f81db77c596492e6f1a5a792ed53
4,e576423831e043f7928d9ac113abbe6f,0e095e054ee94774d6a496099eb1cf6a


## submission

In [12]:
df_submission = pd.read_csv(os.path.join(path, "sample_output_job.csv"))
print(df_submission.shape)
df_submission.head(5)

(8, 1)


,applied
0,0
1,1
2,0
3,1
4,0


In [13]:
df_company.groupby('jobID')['companyID'].value_counts()

jobID                             companyID                       
00ac8ed3b4327bdd4ebbebcb2ba10a00  2dace78f80bc92e6d7493423d729448e    1
00e26af6ac3b1c1c49d7c3d79c60d000  c058f544c737782deacefa532d9add4c    1
00ec53c4682d36f5c4359f4ae7bd7ba1  8757150decbd89b0f5442ca3db4d0e0e    1
013a006f03dbc5392effeb8f18fda755  069d3bb002acd8d7dd095917f9efe4cb    1
0188e8b8b014829e2fa0f430f0a95961  07c5807d0d927dcd0980f86024e5208b    1
                                                                     ..
fed33392d3a48aa149a87a38b875ba4a  512c5cad6c37edb98ae91c8a76c3a291    1
ff4d5fbbafdf976cfdc032e3bde78de5  08419be897405321542838d77f855226    1
ffd52f3c7e12435a724a8f30fddadd9c  4f6ffe13a5d75b2d6a3923922b3922e5    1
ffeabd223de0d4eacb9a3e6e53e5448d  a8e864d04c95572d1aece099af852d0a    1
ffeed84c7cb1ae7bf4ec4bd78275bb98  c0f168ce8900fa56e57789e2a2f2c9d0    1
Name: companyID, Length: 733, dtype: int64

# Data Preprocessing

In [14]:
tags = df_tag['tagID'].unique()
tag_dict = dict()
for i, tag in enumerate(tags):
    tag_dict[tag] = i 

In [15]:
users = df_usertag['userID'].unique()
user_dict = dict()
for i, user in enumerate(users):
    user_dict[user] = i 

In [16]:
jobs = df_jobtag['jobID'].unique()
job_dict = dict()
for i, tag in enumerate(jobs):
    job_dict[tag] = i 

## One Hot Encoded
### Set Tag to One-Hot Encoding Vector

In [17]:
A = np.eye(len(tag_dict))
tag_onehot_dict = dict()
for key, val in tag_dict.items():
    tag_onehot_dict[key] = A[val,:]

### User Technical Stack. Add Tech. Tag Vector

In [18]:
user_stack = dict()

for i in range(df_usertag.shape[0]):
    userID = df_usertag.iloc[i]['userID']
    tagID = df_usertag.iloc[i]['tagID']
    if userID in user_stack.keys():
        user_stack[userID] += tag_onehot_dict[tagID]
    else:
        user_stack[userID] = tag_onehot_dict[tagID].copy()

### Check Stacks

In [19]:
sample = np.random.choice(len(user_stack), 10, replace = False)

for i, (key, val) in enumerate(user_stack.items()):
    if i in sample:
        print("/***************/")
        print(key, val)


/***************/
11bd610a99ecb39c4be8335c6656ac19 [ 0.  0.  0.  0.  0.  0.  0.  0.  9.  0. 12.  0.  0.  0.  0.  9.  0.  0.
 12. 12.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 12.  9.
  0.  0.  9.  0.  0.  0.  0.  0. 12.  0.  0.  0.  0.  0.  0.  0.  0. 12.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 12.  0.  0.
 12.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  9.  0. 12.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  9.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 12.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.


기술 스택은 0, 1로만 이루어진 vector를 예상했지만, 실제로는 1보다 큰 값들이 여러개 포함되어 있다.
### User별 스택 확인

In [20]:
df_usertag.loc[df_usertag['userID'] == 'f69054686ba46877b6397ccdb8f51762', 'tagID'].value_counts()

81dc9bdb52d04dc20036dbd8313ed055    26
20d135f0f28185b84a4cf7aa51f29500    26
2ba8698b79439589fdd2b0f7218d8b07    26
c8ba76c279269b1c6bc8a07e38e78fa4    26
a8c88a0055f636e4a163a5e3d16adab7    26
aace49c7d80767cffec0e513ae886df0    26
4122cb13c7a474c1976c9706ae36521d    26
9996535e07258a7bbfd8b132435c5962    26
f47330643ae134ca204bf6b2481fec47    26
7bccfde7714a1ebadf06c5f4cea752c1    26
e034fb6b66aacc1d48f445ddfb08da98    21
c20ad4d76fe97759aa27a0c99bff6710    21
db2b4182156b2f1f817860ac9f409ad7    21
4e2545f819e67f0615003dd7e04a6087    21
c9f95a0a5af052bffce5c89917335f67    21
26588e932c7ccfa1df309280702fe1b5    21
9bf31c7ff062936a96d3c8bd1f8f2ff3    21
66808e327dc79d135ba18e051673d906    21
1e6e0a04d20f50967c64dac2d639a577    21
82c2559140b95ccda9c6ca4a8b981f1e    21
dd77279f7d325eec933f05b1672f6a1f    21
4c27cea8526af8cfee3be5e183ac9605    21
aff0a6a4521232970b2c1cf539ad0a19     5
8a3363abe792db2d8761d6403605aeb7     5
9adeb82fffb5444e81fa0ce8ad8afe7a     5
7810ccd41bf26faaa2c4e1f20

실제로도 한 유저에게 중복되는 `tagID`가 존재하는 것을 확인. 하지만 동일한 `tagID`가 여럿 있다는 사실 자체는 중요하지 않기 때문에, 기술 스택 유무만 저장 하도록 `user_stack`을 정의하자

In [21]:
for key, val in user_stack.items():
    user_stack[key] = (val > 0).astype(float)

In [22]:
sample = np.random.choice(len(user_stack), 10, replace = False)

for i, (key, val) in enumerate(user_stack.items()):
    if i in sample:
        print("/***************/")
        print(key, val)


/***************/
461beab646d4e7e0fd168125f577483b [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.


### Job Dict

In [23]:
job_dict = dict()

for i in range(df_jobtag.shape[0]):
    jobID = df_jobtag.iloc[i]['jobID']
    tagID = df_jobtag.iloc[i]['tagID']
    if jobID in job_dict.keys():
        job_dict[jobID] += tag_onehot_dict[tagID]
    else:
        job_dict[jobID] = tag_onehot_dict[tagID].copy()

In [24]:
sample = np.random.choice(len(job_dict), 10, replace = False)

for i, (key, val) in enumerate(job_dict.items()):
    if i in sample:
        print("/***************/")
        print(key, val)


/***************/
1e6e0a04d20f50967c64dac2d639a577 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.


In [25]:
def DataFrame2Array(df):
    X = []
    y = []
    for i in range(df.shape[0]):
        userID = df.iloc[i]['userID']
        jobID = df.iloc[i]['jobID']
        
        if 'applied' in df.columns:
            apply = df.iloc[i]['applied']
        else:
            apply = []
        X.append(np.concatenate([user_stack[userID], job_dict[jobID]]).reshape(1, -1))
        y.append(np.array(apply).reshape(1,-1))

    X = np.concatenate(X)
    y = np.concatenate(y)
    
    return X, y

X, y = DataFrame2Array(df_train)
X_test, _ = DataFrame2Array(df_test)

## Data Loader
### First, tarin_val_split

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.1)

In [27]:
class CustomDataset(Dataset):
    def __init__(self, X, y, train = True):
        self.X = torch.Tensor(X)
        self.y = torch.Tensor(y)
        self.len = X.shape[0]
        self.train = train
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        if self.train:
            X, y = self.X[idx, :], self.y[idx,:]
        else:
            X, y = self.X[idx, :], torch.Tensor([])
        return X, y
            
train_dataset = CustomDataset(X_train, y_train)
valid_dataset = CustomDataset(X_valid, y_valid)
test_dataset = CustomDataset(X_test, [], train = False)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [28]:
class FCNEmbedded(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.kaiming_normal_(self.fc3.weight)
        nn.init.kaiming_normal_(self.fc4.weight)
        nn.init.kaiming_normal_(self.fc5.weight)
        
        self.dropout03 = nn.Dropout(0.3)
        self.dropout06 = nn.Dropout(0.6)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout06(x)
        x = F.relu(self.fc2(x))
        x = self.dropout06(x)
        x = F.relu(self.fc3(x))
        x = self.dropout06(x)
        x = F.relu(self.fc4(x))
        x = self.dropout03(x)
        x = self.fc5(x)
        x = self.dropout03(x)
        return x
        


class BaseModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.input_size = input_size
        
        self.fcn = FCNEmbedded(int(input_size/2))
        
        self.fc_out = nn.Linear(1,1)
        
    def forward(self, x):
        size = int(self.input_size/2)
        user = x[:, :size]
        jobs = x[:, size:]
        
        user = self.fcn(user)
        jobs = self.fcn(jobs)
        
        out = F.cosine_similarity(user, jobs)
        out = self.fc_out(out.reshape(-1, 1))
        out = torch.sigmoid(out)
        
        return out, user, jobs

In [29]:
def feature_regularizer(feas):
    d = feas.size()[1]
    batchsize = feas.size()[0]
    feas = feas.reshape(batchsize, d, 1)
    I = torch.eye(d)[None, :, :]
    if feas.is_cuda:
        I = I.cuda()
    loss = torch.mean(torch.norm(torch.bmm(feas, feas.transpose(2,1)) - I, dim=(1,2)))
    return loss

def loss_function(inputs, targets, user, jobs):
    return nn.BCELoss()(inputs, targets) + feature_regularizer(user) + feature_regularizer(jobs)

def score_function(inputs, targets):
    return ((inputs > 0.5) == targets).float().mean()

In [30]:
model = BaseModel(X.shape[1])
model.to(device)

optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                mode = 'min',
                                                factor = 0.1,
                                                patience = 5,
                                                verbose = True)

In [31]:
def train(epoch, progress_log):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    user_append = []
    jobs_append = []
    
    for data, targets in progress_log:
        
        data = data.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        outputs, user, jobs = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_function(outputs, targets, user, jobs)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(outputs)
        ground.append(targets)
        user_append.append(user)
        jobs_append.append(jobs)
        
    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    user_append = torch.cat(user_append,axis = 0)
    jobs_append = torch.cat(jobs_append,axis = 0)
    
    loss = loss_function(predict, ground, user_append, jobs_append)
    score = score_function(predict, ground)
    return loss, score

In [32]:
def valid(progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    user_append = []
    jobs_append = []
    
    with torch.no_grad():
        for data, targets in progress_log:

            data = data.to(device)
            targets = targets.to(device)

            outputs, user, jobs = model(data)  # 데이터를 입력하고 출력을 계산
        
            predict.append(outputs)
            ground.append(targets)
            user_append.append(user)
            jobs_append.append(jobs)
        
    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    user_append = torch.cat(user_append,axis = 0)
    jobs_append = torch.cat(jobs_append,axis = 0)
    
    loss = loss_function(predict, ground, user_append, jobs_append)
    score = score_function(predict, ground)
    return loss, score

In [33]:
def test(progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    with torch.no_grad():
        for data, _ in progress_log:

            data = data.to(device)

            outputs, user, jobs = model(data)  # 데이터를 입력하고 출력을 계산
            
            predict.append(outputs)
        
    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    
    return predict

In [34]:
train_loss_list = []
train_score_list = []
valid_loss_list = []
valid_score_list = []

patience_count = 0
min_valid_score = np.inf
checkpoint_name = ""

if not os.path.isdir(f"./models/{notebookName}/model-{num_files}_checkpoint/"):
    os.mkdir(f"./models/{notebookName}/model-{num_files}_checkpoint/")
    
prog_epoch = tqdm(range(0, nepochs), position = 0, desc = 'EPOCH')
for epoch in prog_epoch:
    print( "-------------------------------------------------------")
    print(f"|EPOCH: {epoch+1}/{nepochs}")
    prog_train = tqdm(train_loader, desc = 'TRAIN', leave = False)
    prog_valid = tqdm(valid_loader, desc = 'VALID', leave = False)

    train_loss, train_score = train(epoch, prog_train)
    valid_loss, valid_score = valid(prog_valid)
    
    scheduler.step(valid_score)
    if valid_score < min_valid_score:
        print(f"|{epoch+1}-th model is checked!, *model-{epoch}-{valid_score}.pth*")
        min_valid_score= valid_score
        checkpoint_name = f"./models/{notebookName}/model-{num_files}_checkpoint/model-{epoch}-{valid_score}.pth"
        torch.save(model.state_dict(), checkpoint_name)
    else:
        patience_count+=1
        if(patience_count > max_patience_count):
            break
    
    train_loss_list.append(train_loss)
    train_score_list.append(train_score)
    valid_loss_list.append(valid_loss)
    valid_score_list.append(valid_score)
    
    print(f"|TRAIN: loss={train_loss:.6f},  score={train_score:.6f}|")
    print(f"|VALID: loss={valid_loss:.6f},  score={valid_score:.6f}|")


history = dict()
history['train_loss'] = train_loss_list
history['train_score'] = train_score_list
history['valid_loss'] = valid_loss_list
history['valid_score'] = valid_score_list

EPOCH:   0%|          | 0/100 [00:00<?, ?it/s]

-------------------------------------------------------
|EPOCH: 1/100


TRAIN:   0%|          | 0/169 [00:00<?, ?it/s]

VALID:   0%|          | 0/19 [00:00<?, ?it/s]

|1-th model is checked!, *model-0-0.8250000476837158.pth*
|TRAIN: loss=17.489410,  score=0.665926|
|VALID: loss=16.483175,  score=0.825000|
-------------------------------------------------------
|EPOCH: 2/100


...

VALID:   0%|          | 0/19 [00:00<?, ?it/s]

|TRAIN: loss=16.285408,  score=0.860741|
|VALID: loss=16.360315,  score=0.825000|
-------------------------------------------------------
|EPOCH: 38/100


TRAIN:   0%|          | 0/169 [00:00<?, ?it/s]

VALID:   0%|          | 0/19 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 8.00 GiB total capacity; 6.41 GiB already allocated; 0 bytes free; 6.43 GiB reserved in total by PyTorch)

In [ ]:
history = dict()
history['train_loss'] = train_loss_list
history['train_score'] = train_score_list
history['valid_loss'] = valid_loss_list
history['valid_score'] = valid_score_list

In [ ]:
plt.figure(figsize = (16,6))
plt.subplot(2,1,1)
plt.plot(history['train_loss'], label = 'train')
plt.plot(history['valid_loss'], label = 'valid')
plt.ylabel('loss')

plt.subplot(2,1,2)
plt.plot(history['train_score'], label = 'train')
plt.plot(history['valid_score'], label = 'valid')
plt.ylabel('score')

In [ ]:
predict = test(test_loader)
print(predict.shape)
predict = (predict > 0.5).int().to('cpu').detach().numpy().reshape(-1)
predict

In [ ]:
submission = pd.DataFrame([])
submission['applied'] = predict

print(submission.shape)
print(submission.mean())
submission.sample(10)

In [ ]:
submission.to_csv(f"{notebookName}-submission.csv", index = False)